# Equities Futures and Options traded on the NSE

The python version of this sample can be found [here](https://github.com/shyams80/plutons/blob/master/docs-py/EquitiesFuturesAndOptionsIndiaNse.ipynb)

In [1]:
library(dbplyr)
library(dplyr)
library(odbc)
library(plutoR)
options("scipen"=999)
source("config.R")

# initialize
eqFo <- EquitiesFuturesAndOptionsIndiaNse()


Attaching package: ‘dplyr’

The following objects are masked from ‘package:dbplyr’:

    ident, sql

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



### get all NIFTY futures contract traded right now

In [2]:
endDt <- (eqFo$FuturesEodTimeSeries() %>%
            summarize(MAX_TS = max(TIME_STAMP)) %>%
            collect())$MAX_TS[1]

eqFo$FuturesEodTimeSeries() %>%
  filter(SYMBOL == "NIFTY" & TIME_STAMP == endDt) %>%
  print()

Warning message:
“Missing values are always removed in SQL.
Use `MAX(x, na.rm = TRUE)` to silence this warning
This warning is displayed only once per session.”

# Source:   lazy query [?? x 11]
# Database: Microsoft SQL Server 13.00.4224[ro1@NORWAY/StockViz]
  TIME_STAMP SYMBOL EXPIRY       HIGH   LOW   OPEN  CLOSE SETTLE CONTRACTS
  <date>     <chr>  <date>      <dbl> <dbl>  <dbl>  <dbl>  <dbl>     <int>
1 2019-07-19 NIFTY  2019-07-25 11643. 11402 11635. 11422. 11422.    160198
2 2019-07-19 NIFTY  2019-08-29 11682. 11444 11675. 11461. 11461.     19739
3 2019-07-19 NIFTY  2019-09-26 11740. 11500 11722. 11517. 11517.      3078
# … with 2 more variables: VALUE <dbl>, OI <integr64>


### get all NIFTY option contracts at the nearest expiry traded right now

In [3]:
endDt <- (eqFo$OptionsEodTimeSeries() %>%
            summarize(MAX_TS = max(TIME_STAMP)) %>%
            collect())$MAX_TS[1]

expiry <- (eqFo$OptionsEodTimeSeries() %>%
             filter(SYMBOL == "NIFTY" & TIME_STAMP == endDt) %>%
             summarize(MIN_TS = min(EXPIRY)) %>%
             collect())$MIN_TS[1]

eqFo$OptionsEodTimeSeries() %>%
  filter(SYMBOL == "NIFTY" & TIME_STAMP == endDt
         & EXPIRY == expiry & as.integer(STRIKE) %% 100 == 0) %>%
  arrange(STRIKE, OTYPE) %>%
  print()

Warning message:
“Missing values are always removed in SQL.
Use `MIN(x, na.rm = TRUE)` to silence this warning
This warning is displayed only once per session.”

# Source:     lazy query [?? x 13]
# Database:   Microsoft SQL Server 13.00.4224[ro1@NORWAY/StockViz]
# Ordered by: STRIKE, OTYPE
   TIME_STAMP SYMBOL EXPIRY     STRIKE OTYPE    HIGH     LOW    OPEN   CLOSE
   <date>     <chr>  <date>      <dbl> <chr>   <dbl>   <dbl>   <dbl>   <dbl>
 1 2019-07-19 NIFTY  2019-07-25   9700 CE    1928    1692    1922.   1710.  
 2 2019-07-19 NIFTY  2019-07-25   9700 PE       1       0.45    1       0.6 
 3 2019-07-19 NIFTY  2019-07-25   9800 CE    1770.   1654    1770.   1658   
 4 2019-07-19 NIFTY  2019-07-25   9800 PE       1.75    0.1     0.95    0.75
 5 2019-07-19 NIFTY  2019-07-25   9900 CE    1725.   1500    1725.   1508.  
 6 2019-07-19 NIFTY  2019-07-25   9900 PE       2.5     0.35    2.5     1.15
 7 2019-07-19 NIFTY  2019-07-25  10000 CE    1617.   1396.   1617.   1413.  
 8 2019-07-19 NIFTY  2019-07-25  10000 PE       0.95    0.5     0.5     0.85
 9 2019-07-19 NIFTY  2019-07-25  10100 CE    1397.   1329.   1397.   1336.  
10 2019-07-19 NIFTY  20

### get all greeks for the NIFTY option contracts at the nearest expiry traded right now

In [4]:
endDt <- (eqFo$OptionGreeks() %>%
            summarize(MAX_TS = max(TIME_STAMP)) %>%
            collect())$MAX_TS[1]

expiry <- (eqFo$OptionGreeks() %>%
             filter(SYMBOL == "NIFTY" & TIME_STAMP == endDt) %>%
             summarize(MIN_TS = min(EXPIRY)) %>%
             collect())$MIN_TS[1]

eqFo$OptionGreeks() %>%
  filter(SYMBOL == "NIFTY" & TIME_STAMP == endDt
         & EXPIRY == expiry & as.integer(STRIKE) %% 100 == 0) %>%
  arrange(STRIKE, OTYPE) %>%
  print()


# Source:     lazy query [?? x 16]
# Database:   Microsoft SQL Server 13.00.4224[ro1@NORWAY/StockViz]
# Ordered by: STRIKE, OTYPE
   TIME_STAMP SYMBOL EXPIRY     STRIKE OTYPE MODEL_PRICE     DELTA    THETA
   <date>     <chr>  <date>      <dbl> <chr>       <dbl>     <dbl>    <dbl>
 1 2019-07-19 NIFTY  2019-07-25   9700 CE       1.73e+ 3 10.00e- 1 -6.35e+2
 2 2019-07-19 NIFTY  2019-07-25   9700 PE       7.82e-11 -2.11e-12 -1.21e-7
 3 2019-07-19 NIFTY  2019-07-25   9800 CE       1.63e+ 3 10.00e- 1 -6.42e+2
 4 2019-07-19 NIFTY  2019-07-25   9800 PE       1.60e- 9 -4.08e-11 -2.19e-6
 5 2019-07-19 NIFTY  2019-07-25   9900 CE       1.53e+ 3 10.00e- 1 -6.49e+2
 6 2019-07-19 NIFTY  2019-07-25   9900 PE       2.68e- 8 -6.39e-10 -3.22e-5
 7 2019-07-19 NIFTY  2019-07-25  10000 CE       1.43e+ 3 10.00e- 1 -6.55e+2
 8 2019-07-19 NIFTY  2019-07-25  10000 PE       3.67e- 7 -8.17e- 9 -3.84e-4
 9 2019-07-19 NIFTY  2019-07-25  10100 CE       1.33e+ 3 10.00e- 1 -6.62e+2
10 2019-07-19 NIFTY  2019-07-25  1

In [5]:
### get NIFTY's lot-sizes at different expiries

In [6]:
futureDate <- as.Date(Sys.Date() + 30*3)

eqFo$LotSize() %>%
  filter(SYMBOL == "NIFTY" & CONTRACT <= futureDate) %>%
  top_n(10, wt = CONTRACT) %>%
  print()

# Source:   lazy query [?? x 3]
# Database: Microsoft SQL Server 13.00.4224[ro1@NORWAY/StockViz]
   SYMBOL CONTRACT   LOT_SIZE
   <chr>  <date>        <int>
 1 NIFTY  2019-09-01       75
 2 NIFTY  2019-08-01       75
 3 NIFTY  2019-07-01       75
 4 NIFTY  2019-06-01       75
 5 NIFTY  2019-05-01       75
 6 NIFTY  2019-04-01       75
 7 NIFTY  2019-03-01       75
 8 NIFTY  2019-02-01       75
 9 NIFTY  2019-01-01       75
10 NIFTY  2018-12-01       75


This notebook was created using [pluto](http://pluto.studio). Learn more [here](https://github.com/shyams80/pluto)